In [1]:
import pandas as pd
import altair as alt
import numpy as np
import openpyxl
import math
import matplotlib.pyplot as plt

In [5]:
# Load and clean data
expenditure_by_party_df = pd.read_csv('../Data/Clean/from_sql/expenditure_by_party.csv')
expenditure_by_party_df = expenditure_by_party_df.iloc[2:-1]
expenditure_by_party_df.columns = ['province', 'year', 'party', 'percent_expenditure_change']
expenditure_by_party_df

,province,year,party,percent_expenditure_change
2,Alberta,2000,Conservative Party,10.2
3,BritishColumbia,2000,NDP,9.2
4,Ontario,2000,Conservative Party,10.9
5,Alberta,2001,Conservative Party,13.2
6,BritishColumbia,2001,NDP,10.4
...,...,...,...,...
72,BritishColumbia,2023,NDP,8.9
73,Ontario,2023,Conservative Party,2.1
74,Alberta,2024,Conservative Party,5
75,BritishColumbia,2024,NDP,12
